In [1]:
!pip install python-telegram-bot --upgrade

In [4]:
import random
import nltk
import json
import io
import logging
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

In [5]:
with io.open('/content/BOT_CONFIG.json', 'r') as f:
  BOT_CONFIG = json.load(f)

In [6]:
X = []
y = []

for intent in BOT_CONFIG['intents'].keys():
    try:
      if intent != 'other':
        for example in BOT_CONFIG['intents'][intent]['examples']:
            X.append(example)
            y.append(intent)
    except:
      print(BOT_CONFIG['intents'][intent])

In [ ]:
len(X), len(y)

(1173, 1173)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [ ]:
vectorizer = CountVectorizer(ngram_range=(2, 4), analyzer='char')
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)
len(vectorizer.get_feature_names())

6762

In [9]:
clf = LogisticRegression().fit(X_train_vectorized, y_train)
# clf = RandomForestClassifier(n_estimators=200).fit(X_train_vectorized, y_train)

In [10]:
clf.score(X_train_vectorized, y_train)

0.9821428571428571

In [ ]:
clf.score(X_test_vectorized, y_test)

0.8571428571428571

In [ ]:
clf.predict(vectorizer.transform(['откуда ты?']))

array(['whereareyoufrom'], dtype='<U22')

In [11]:
def get_intent_by_model(text):
  return clf.predict(vectorizer.transform([text]))[0]

## Вызов бота

In [12]:
def bot(input_text):
  intent = get_intent_by_model(input_text)
  return random.choice(BOT_CONFIG['intents'][intent]['responces'])

In [16]:
# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)


# Define a few command handlers. These usually take the two arguments update and
# context.
def start(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True),
    )


def help_command(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def echo(update: Update, context: CallbackContext) -> None:
    """Echo the user message."""
    update.message.reply_text(bot(update.message.text))


def main() -> None:
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater("1968328824:AAE5qKVIsTi2cWl3pnLLb3yvPdCEyjV17h8")

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))

    # on non command i.e message - echo the message on Telegram
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()

In [17]:
main()

2021-09-08 19:24:38,923 - apscheduler.scheduler - INFO - Scheduler started
2021-09-08 19:27:13,868 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
2021-09-08 19:27:13,873 - apscheduler.scheduler - INFO - Scheduler has been shut down
